In [61]:
import geopandas as gpd
import folium
import overpy
import pandas as pd
from datetime import datetime
import json
import os

In [59]:
# load administrative boundaries
boundaries_cities = gpd.read_file('https://storage.googleapis.com/data_portal_exposure/data/administrative_boundaries/mapped/boundary_CHL_vitacura_mapped_geom.geojson')


In [9]:
# add centroid coordinates
boundaries_cities["centroid_x"] = boundaries_cities.centroid.x
boundaries_cities["centroid_y"] = boundaries_cities.centroid.y

C:\Users\SAIF~1.SHA\AppData\Local\Temp/ipykernel_16704/1932643050.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  boundaries_cities["centroid_x"] = boundaries_cities.centroid.x
C:\Users\SAIF~1.SHA\AppData\Local\Temp/ipykernel_16704/1932643050.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  boundaries_cities["centroid_y"] = boundaries_cities.centroid.y


# Get amenities

In [37]:
city_name = 'Vitacura'

In [39]:
def get_osm_amenity(city_name):
    print("\n Get Amenity")
    # get city geometry
    gdf_city = boundaries_cities[boundaries_cities.city_name == city_name]

    # get boudary box
    xmin,ymin,xmax,ymax = gdf_city.total_bounds
    city_bbox = "("+ str(ymin)+","+str(xmin) + ","+str(ymax)+ "," + str(xmax)+ ")"
    print(city_name+" city_bbox: "+city_bbox)

    # build query
    query_prefix = """[out:json][timeout:250];(node["amenity"]"""
    query_bbox = city_bbox
    query_suffix = ";);out;"""
    built_query = query_prefix+query_bbox+query_suffix
    print(built_query) 

    # get results
    api = overpy.Overpass()  
    result = api.query(built_query)

    # store result in a dataframe
    list_of_node_tags = []                        # initializing empty list , we'll use it to form a dataframe .
    for node in result.nodes: 
        node.tags['latitude'] =  node.lat
        node.tags['longitude'] = node.lon
        node.tags['id'] = node.id
        list_of_node_tags.append(node.tags)
    data_frame = pd.DataFrame(list_of_node_tags)

    # rename columns
    data_frame.rename(columns = {'amenity':'featureType',
                                 'name':'featureName'}, 
                      inplace = True)

    # add column
    data_frame['featureCategory'] = 'amenity'

    # select columns
    data_frame = data_frame[['featureCategory',
                           'featureType',
                           'featureName',
                           'id',
                           'latitude',
                           'longitude']]
    
    print(city_name+ " amenity data created")
    
    return data_frame                             # return data frame if you want to use it further in main function.
    

In [43]:
def get_osm_building(city_name):
    print("\n Get Building")
    # get city geometry
    gdf_city = boundaries_cities[boundaries_cities.city_name == city_name]

    # get boudary box
    xmin,ymin,xmax,ymax = gdf_city.total_bounds
    city_bbox = "("+ str(ymin)+","+str(xmin) + ","+str(ymax)+ "," + str(xmax)+ ")"
    print(city_name+" city_bbox: "+city_bbox)

    # build query
    query_prefix = """[out:json][timeout:250];(node["building"]"""
    query_bbox = city_bbox
    query_suffix = ";);out;"""
    built_query = query_prefix+query_bbox+query_suffix
    print(built_query) 

    # get results
    api = overpy.Overpass()  
    result = api.query(built_query)

    # store result in a dataframe
    list_of_node_tags = []                        # initializing empty list , we'll use it to form a dataframe .
    for node in result.nodes: 
        node.tags['latitude'] =  node.lat
        node.tags['longitude'] = node.lon
        node.tags['id'] = node.id
        list_of_node_tags.append(node.tags)
    data_frame = pd.DataFrame(list_of_node_tags)

    # rename columns
    data_frame.rename(columns = {'building':'featureType',
                                 'name':'featureName'}, 
                      inplace = True)

    # add column
    data_frame['featureCategory'] = 'building'

    # select columns
    data_frame = data_frame[['featureCategory',
                           'featureType',
                           'featureName',
                           'id',
                           'latitude',
                           'longitude']]
    
    print(city_name+ " building data created")
    
    return data_frame                             # return data frame if you want to use it further in main function.
    

In [48]:
def get_osm_city(city_name):
    """ Call the  get_osm_amenity and get_osm_building functions and combine the results 
    into one data frame by renaming the columns

        Parameters:
         city_name (str): the name of the city that should exist in the boundaries cities geojson file
    
        Returns:
         data_frame (pandas.DataFrame): df containong the amenities and buildings within the administrative boundaries

         Enhancement:
         - Add dataportal cities geojson file as parameter
         - put the extraction of bbox here to avoid repeating in the two previous functions
         - specify a data model
    """
    # get amenity data
    aminety_city = get_osm_amenity(city_name = city_name)
    # get building data
    building_city = get_osm_building(city_name = city_name)
    # append data
    osm_city = pd.concat([aminety_city, building_city])
    # add column
    osm_city['cityName'] = city_name
    osm_city['integrationDate'] = datetime.today().strftime('%Y-%m-%d')
    osm_city['objectType'] = 'amenity'
    osm_city['projectName'] = 'dataportal'
    osm_city = osm_city[['featureCategory','featureType','id','latitude','longitude','cityName','integrationDate','objectType','projectName']]
    # convert lon lat to numeric
    osm_city["latitude"] = pd.to_numeric(osm_city["latitude"])
    osm_city["longitude"] = pd.to_numeric(osm_city["longitude"])
    print("data concatenated")
    # output
    return(osm_city)

In [51]:
osm_city_amenities = get_osm_city(city_name = city_name)
osm_city_amenities.head()


 Get Amenity
Vitacura city_bbox: (-33.4092505,-70.6134731,-33.3507499,-70.5176045)
[out:json][timeout:250];(node["amenity"](-33.4092505,-70.6134731,-33.3507499,-70.5176045););out;
Vitacura amenity data created

 Get Building
Vitacura city_bbox: (-33.4092505,-70.6134731,-33.3507499,-70.5176045)
[out:json][timeout:250];(node["building"](-33.4092505,-70.6134731,-33.3507499,-70.5176045););out;
Vitacura building data created
data concatenated


,featureCategory,featureType,id,latitude,longitude,cityName,integrationDate,objectType,projectName
0,amenity,pharmacy,240434655,-33.376238,-70.525053,Vitacura,2021-09-23,amenity,dataportal
1,amenity,parking,253242064,-33.393896,-70.600037,Vitacura,2021-09-23,amenity,dataportal
2,amenity,parking,253272975,-33.402910,-70.580557,Vitacura,2021-09-23,amenity,dataportal
3,amenity,parking,253276941,-33.392850,-70.575862,Vitacura,2021-09-23,amenity,dataportal
4,amenity,parking,253433629,-33.403386,-70.579463,Vitacura,2021-09-23,amenity,dataportal


In [73]:
# convert to geojson
osm_city_gdf = gpd.GeoDataFrame(
  osm_city_amenities, geometry=gpd.points_from_xy(osm_city_amenities.longitude, osm_city_amenities.latitude))
osm_city_geojson = osm_city_gdf.to_json()

In [76]:
osm_city_gdf.head()

,featureCategory,featureType,id,latitude,longitude,cityName,integrationDate,objectType,projectName,geometry
0,amenity,pharmacy,240434655,-33.376238,-70.525053,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.52505 -33.37624)
1,amenity,parking,253242064,-33.393896,-70.600037,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.60004 -33.39390)
2,amenity,parking,253272975,-33.402910,-70.580557,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.58056 -33.40291)
3,amenity,parking,253276941,-33.392850,-70.575862,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.57586 -33.39285)
4,amenity,parking,253433629,-33.403386,-70.579463,Vitacura,2021-09-23,amenity,dataportal,POINT (-70.57946 -33.40339)


# Store results

## Store local

In [77]:
# export local
osm_city_gdf.to_file("amenity_Vitacura.geojson", driver='GeoJSON')

## Store in GCS

In [67]:
from google.cloud import storage
from google.oauth2 import service_account

In [65]:
# set key
with open('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\Data portal\\github\\cities-dataportal-exposure\\key\\wri-gee-key.json') as source:
    info = json.load(source)

In [68]:
# set credentials
storage_credentials = service_account.Credentials.from_service_account_info(info)

In [71]:
# instatiate a google storage client and specify reauired bucket and file
storgae_client = storage.Client(project="wri-gee",credentials=storage_credentials)
bucket = storgae_client.get_bucket('data_portal_exposure')

Forbidden: 403 GET https://storage.googleapis.com/storage/v1/b/data_portal_exposure?projection=noAcl&prettyPrint=false: dataportal@wri-gee.iam.gserviceaccount.com does not have storage.buckets.get access to the Google Cloud Storage bucket.

# Plot

In [18]:
############################### plot a map of geoboundaries

# define map center coordinates
map_center_x = boundaries_cities.centroid_x.mean()
map_center_y = boundaries_cities.centroid_y.mean()

# plot map background
m = folium.Map()
m = folium.Map(location = [map_center_y,map_center_x],zoom_start=10)

# plot adminisrative boundaries
folium.GeoJson(
    boundaries_cities,
    tooltip=folium.features.GeoJsonTooltip(fields=['country_iso3','city_name'],
            aliases=['Country','City name'],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
            sticky=True
        )
).add_to(m)

m

In [52]:
# plot amenities

In [53]:
from folium.plugins import MarkerCluster
MarkerCluster()

In [54]:
locations = osm_city_amenities[['latitude', 'longitude']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

[-33.3739911, -70.5197841]

In [55]:
# plot map
map = folium.Map(location=[-33.3739911, -70.5197841], zoom_start=12)
marker_cluster = MarkerCluster().add_to(map)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point]).add_to(marker_cluster)
map